In [1]:
import pickle
import numpy as np

In [2]:
with open("train_qa.txt", "rb") as fp:   # Unpickling
    train_data =  pickle.load(fp)

In [3]:
with open("test_qa.txt", "rb") as fp:   # Unpickling
    test_data =  pickle.load(fp)

In [4]:
type(test_data)

list

In [5]:
type(train_data)

list

In [6]:
len(test_data)

1000

In [7]:
len(train_data)

10000

In [8]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [9]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [10]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [11]:
train_data[0][2]

'no'

In [12]:
# Create a set that holds the vocab words
vocab = set()

In [13]:
all_data = test_data + train_data

In [14]:
for story, question , answer in all_data:
    # In case you don't know what a union of sets is:
    # https://www.programiz.com/python-programming/methods/set/union
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [15]:
vocab.add('no')
vocab.add('yes')

In [16]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [17]:
vocab_len = len(vocab) + 1 #we add an extra space to hold a 0 for Keras's pad_sequences

In [18]:
max_story_len = max([len(data[0]) for data in all_data])

In [19]:
max_story_len

156

In [20]:
max_question_len = max([len(data[1]) for data in all_data])

In [21]:
max_question_len

6

In [26]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [27]:
# Reserve 0 for pad_sequences
vocab_size = len(vocab) + 1

In [28]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [29]:
# integer encode sequences of words
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [30]:
tokenizer.word_index

{'kitchen': 1,
 'is': 2,
 '.': 3,
 'office': 4,
 'journeyed': 5,
 'apple': 6,
 'up': 7,
 'moved': 8,
 'travelled': 9,
 'picked': 10,
 'no': 11,
 'yes': 12,
 'bathroom': 13,
 'in': 14,
 'bedroom': 15,
 'went': 16,
 'football': 17,
 'garden': 18,
 'hallway': 19,
 'got': 20,
 'grabbed': 21,
 'down': 22,
 'mary': 23,
 'to': 24,
 'put': 25,
 'back': 26,
 'john': 27,
 'the': 28,
 'there': 29,
 'dropped': 30,
 'milk': 31,
 'discarded': 32,
 'left': 33,
 'daniel': 34,
 '?': 35,
 'took': 36,
 'sandra': 37}

In [31]:
train_story_text = []
train_question_text = []
train_answers = []

for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)

In [32]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [33]:
len(train_story_text)

10000

In [34]:
len(train_story_seq)

10000

In [35]:
# word_index = tokenizer.word_index

In [36]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_question_len):
    '''
    INPUT: 
    
    data: consisting of Stories,Queries,and Answers
    word_index: word index dictionary from tokenizer
    max_story_len: the length of the longest story (used for pad_sequences function)
    max_question_len: length of the longest question (used for pad_sequences function)


    OUTPUT:
    
    Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
    answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
    output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
    
    Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
    '''
    
    
    # X = STORIES
    X = []
    # Xq = QUERY/QUESTION
    Xq = []
    # Y = CORRECT ANSWER
    Y = []
    
    
    for story, query, answer in data:
        
        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        # Grab the word index for every word in query
        xq = [word_index[word.lower()] for word in query]
        
        # Grab the Answers (either Yes/No so we don't need to use list comprehension here)
        # Index 0 is reserved so we're going to use + 1
        y = np.zeros(len(word_index) + 1)
        
        # Now that y is all zeros and we know its just Yes/No , we can use numpy logic to create this assignment
        #
        y[word_index[answer]] = 1
        
        # Append each set of story,query, and answer to their respective holding lists
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    # Finally, pad the sequences based on their max length so the RNN can be trained on uniformly long sequences.
        
    # RETURN TUPLE FOR UNPACKING
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [37]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [38]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [39]:
inputs_test

array([[ 0,  0,  0, ..., 28, 15,  3],
       [ 0,  0,  0, ..., 28, 18,  3],
       [ 0,  0,  0, ..., 28, 18,  3],
       ...,
       [ 0,  0,  0, ..., 28,  6,  3],
       [ 0,  0,  0, ..., 28, 18,  3],
       [ 0,  0,  0, ...,  6, 29,  3]])

In [40]:
queries_test

array([[ 2, 27, 14, 28,  1, 35],
       [ 2, 27, 14, 28,  1, 35],
       [ 2, 27, 14, 28, 18, 35],
       ...,
       [ 2, 23, 14, 28, 15, 35],
       [ 2, 37, 14, 28, 18, 35],
       [ 2, 23, 14, 28, 18, 35]])

In [41]:
answers_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [42]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
       503., 497.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

In [43]:
tokenizer.word_index['yes']

12

In [44]:
tokenizer.word_index['no']

11

In [46]:
from keras.models import Sequential, Model
from keras.layers import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM

In [47]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [48]:
# Input gets embedded to a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

# This encoder will output:
# (samples, story_maxlen, embedding_dim)

In [49]:
# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

In [50]:
# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=max_question_len))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

In [51]:
# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [52]:
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

In [53]:
# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

In [54]:
# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

In [55]:
answer

<KerasTensor: shape=(None, 6, 220) dtype=float32 (created by layer 'concatenate')>

In [56]:
# Reduce with RNN (LSTM)
answer = LSTM(32)(answer)  # (samples, 32)

In [57]:
# Regularization with Dropout
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)

In [58]:
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 156)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 6)]                  0         []                            
                                                                                                  
 sequential (Sequential)     (None, None, 64)             2432      ['input_1[0][0]']             
                                                                                                  
 sequential_2 (Sequential)   (None, 6, 64)                2432      ['input_2[0][0]']             
                                                                                              

In [60]:
# train
history = model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=120,validation_data=([inputs_test, queries_test], answers_test))

Epoch 1/120
313/313 [==============================] - 9s 19ms/step - loss: 0.8810 - accuracy: 0.5060 - val_loss: 0.6954 - val_accuracy: 0.4970
Epoch 2/120
313/313 [==============================] - 5s 16ms/step - loss: 0.7075 - accuracy: 0.4905 - val_loss: 0.6936 - val_accuracy: 0.4970
Epoch 3/120
313/313 [==============================] - 5s 15ms/step - loss: 0.6973 - accuracy: 0.5025 - val_loss: 0.6972 - val_accuracy: 0.4970
Epoch 4/120
313/313 [==============================] - 4s 14ms/step - loss: 0.6969 - accuracy: 0.4972 - val_loss: 0.6933 - val_accuracy: 0.4970
Epoch 5/120
313/313 [==============================] - 5s 16ms/step - loss: 0.6953 - accuracy: 0.5051 - val_loss: 0.6933 - val_accuracy: 0.4970
Epoch 6/120
313/313 [==============================] - 5s 16ms/step - loss: 0.6958 - accuracy: 0.4944 - val_loss: 0.6935 - val_accuracy: 0.4970
Epoch 7/120
313/313 [==============================] - 5s 14ms/step - loss: 0.6948 - accuracy: 0.5059 - val_loss: 0.6933 - val_accuracy:

313/313 [==============================] - 6s 20ms/step - loss: 0.6367 - accuracy: 0.6427 - val_loss: 0.6249 - val_accuracy: 0.6560
Epoch 58/120
313/313 [==============================] - 7s 21ms/step - loss: 0.6329 - accuracy: 0.6535 - val_loss: 0.6227 - val_accuracy: 0.6500
Epoch 59/120
313/313 [==============================] - 6s 20ms/step - loss: 0.6297 - accuracy: 0.6507 - val_loss: 0.6179 - val_accuracy: 0.6490
Epoch 60/120
313/313 [==============================] - 6s 20ms/step - loss: 0.6239 - accuracy: 0.6540 - val_loss: 0.6176 - val_accuracy: 0.6430
Epoch 61/120
313/313 [==============================] - 6s 20ms/step - loss: 0.6167 - accuracy: 0.6605 - val_loss: 0.6093 - val_accuracy: 0.6530
Epoch 62/120
313/313 [==============================] - 6s 20ms/step - loss: 0.6137 - accuracy: 0.6636 - val_loss: 0.6129 - val_accuracy: 0.6600
Epoch 63/120
313/313 [==============================] - 7s 23ms/step - loss: 0.6083 - accuracy: 0.6658 - val_loss: 0.6101 - val_accuracy: 0.655

313/313 [==============================] - 6s 20ms/step - loss: 0.3785 - accuracy: 0.8306 - val_loss: 0.4156 - val_accuracy: 0.8170
Epoch 114/120
313/313 [==============================] - 6s 20ms/step - loss: 0.3713 - accuracy: 0.8389 - val_loss: 0.4145 - val_accuracy: 0.8060
Epoch 115/120
313/313 [==============================] - 6s 20ms/step - loss: 0.3712 - accuracy: 0.8378 - val_loss: 0.4146 - val_accuracy: 0.8090
Epoch 116/120
313/313 [==============================] - 6s 20ms/step - loss: 0.3707 - accuracy: 0.8367 - val_loss: 0.4286 - val_accuracy: 0.8150
Epoch 117/120
313/313 [==============================] - 6s 20ms/step - loss: 0.3627 - accuracy: 0.8429 - val_loss: 0.4044 - val_accuracy: 0.8140
Epoch 118/120
313/313 [==============================] - 6s 20ms/step - loss: 0.3633 - accuracy: 0.8373 - val_loss: 0.4285 - val_accuracy: 0.8070
Epoch 119/120
313/313 [==============================] - 6s 20ms/step - loss: 0.3617 - accuracy: 0.8414 - val_loss: 0.4188 - val_accuracy:

In [80]:
filename = 'chatbot'
model.save(filename)

INFO:tensorflow:Assets written to: chatbot\assets


INFO:tensorflow:Assets written to: chatbot\assets


In [81]:
model.load_weights(filename)
pred_results = model.predict(([inputs_test, queries_test]))

32/32 [==============================] - 0s 4ms/step


In [82]:
test_data[0][0]

['Mary',
 'got',
 'the',
 'milk',
 'there',
 '.',
 'John',
 'moved',
 'to',
 'the',
 'bedroom',
 '.']

In [83]:
story =' '.join(word for word in test_data[0][0])
print(story)

Mary got the milk there . John moved to the bedroom .


In [84]:
query = ' '.join(word for word in test_data[0][1])
print(query)

Is John in the kitchen ?


In [85]:
print("True Test Answer from Data is:",test_data[0][2])

True Test Answer from Data is: no


In [86]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  no
Probability of certainty was:  0.9961104


In [87]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [99]:
# Note the whitespace of the periods
my_story = "John left the kitchen . Sandra dropped the football in the garden ."
my_story.split()

['John',
 'left',
 'the',
 'kitchen',
 '.',
 'Sandra',
 'dropped',
 'the',
 'football',
 'in',
 'the',
 'garden',
 '.']

In [100]:
my_question = "Is the football in the garden ?"

In [101]:
my_question.split()

['Is', 'the', 'football', 'in', 'the', 'garden', '?']

In [102]:
mydata = [(my_story.split(),my_question.split(),'yes')]

In [103]:
my_story,my_ques,my_ans = vectorize_stories(mydata)

In [94]:
pred_results = model.predict(([ my_story, my_ques]))

1/1 [==============================] - 0s 26ms/step


In [96]:
#Generate prediction from model
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  yes
Probability of certainty was:  0.8018828


In [104]:
with open('chatbot_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

In [105]:
pickle.dump(tokenizer,open('tokenizer.pkl','wb'))